# Medium range from the RCP destination

This notebook will show you:
- how to find which dates and steps to use to download the re-forecacsts - 6-hourly steps from medium range re-forecasts
- download the re-forecasts
- calculate daily mean/max/min/sum of the parameter
- calculate the percentiles, mean and standard deviation of the fields for each day
- calculate the percentiles for each day

Please note that the climate built using this notebook is only valid for the date it was built. If you want to build the climate on some other day, you need to download the data again with the correct dates and steps.

If you want to build the climate every day, you may download all the available steps at once and reuse the files.

For this notebook to work, we will need the libraries from the next cell.

TO DO: add the list here and put the links to the documentation

In [1]:
import datetime
from earthkit.time import Sequence, model_climate_dates, date_range, DailySequence
import requests
import metview as mv
import os

Sequenece is Abstract representation of a sequence of dates.  
For the re-forecasts we use either 'ecmwf-4days' (medium) or ecmwf-2days (sub-seasonal) sequence. These are built in, and represent configuration of ECMWF forecast systems.

In [2]:
sequence = Sequence.from_resource("ecmwf-4days")
sequence

MonthlySequence(days=[1, 5, 9, 13, 17, 21, 25, 29], excludes={(2, 29)})

Next thing we need is to know what dates we need to build the model climate for today's forecast.  
Form medium range model climate (M climate) we can get this using the model_climate_dates function.

model_climate_dates(reference: date, start: date | int, end: date | int, before: timedelta | int, after: timedelta | int, sequence: Sequence)→ Iterator[date]  
Parameters:
- `reference (datetime.date)` – Reference date for the climate
- `start (datetime.date or int)` – Start of the climatological period. Either a full date or a year
- `end (datetime.date or int)` – End of the climatological period. Either a full date or a year
- `before (datetime.timedelta or int)` – Cut-off before the reference date. Either a timedelta or a number of days
- `after (datetime.timedelta or int)` – Cut-off after the reference date. Either a timedelta or a number of days
- `sequence (earthkit.time.sequence.Sequence)` – Sequence of available dates in the reference set

Returns: Sequence of dates

Now we have a sequence, we can find a closest reforecast date for today, and from there, all the reforecast dates we need to calculate the model climate.

In [3]:
today = datetime.date.today()
today

datetime.date(2024, 9, 11)

Now we need to calculate the closest day of the ECMWF re-forecasts.  
We will use the **nearest** funciton.

In [4]:
clim_date = sequence.nearest(today)
clim_date

datetime.date(2024, 9, 9)

Next we will calculate the sequence of 9 dates, around our climatology date (including the climatology date).  
For this we can use the bracket function.  
We need to give it `clim_date`, number of days around `clim_date` we want reforecasts for and we need to set `strict` to False to include the `clim_date` in the set of dates.

In [5]:
clim_dates = sequence.bracket(clim_date,4,strict=False)
clim_dates

<generator object Sequence.bracket at 0x1113c9ad0>

We can loop through the `clim_dates` to see what we got:

In [6]:
for c in clim_dates:
    print(c)

2024-08-25
2024-08-29
2024-09-01
2024-09-05
2024-09-09
2024-09-13
2024-09-17
2024-09-21
2024-09-25


We have prepared a test set of reforecasts available at the address: https://xdiss.ecmwf.int/ecpds/home/rcp/  
From here users can download medium and sub-seasonal range re-forecasts to test their system.  
**The data here is available strictly for testing and the operational and commercial use is not allowed.**

Before 11 September we were uploading all available parameters to this system, however this proved to be challenging for users due to volume of the data. Therefore, since 13 September we have reduced the number of available parameters to those that we produce the M-climate for.

In [7]:
clim_dates = sequence.bracket(clim_date,4,strict=False)
for date in clim_dates:
    month, day = date.month, date.day
    print(month,day)
    ref_date = date.strftime("%Y%m%d")
    month_day_str = date.strftime("%m%d")
    forecast_seq = DailySequence()
    fdate = date
    for i in range(7):        
        month_day_str_f = fdate.strftime("%m%d")
        url = f'https://xdiss.ecmwf.int/ecpds/home/rcp/{ref_date}/mediumrange/A1I{month_day_str}0000{month_day_str_f}____0079.RCP91DG1000001' 
        print(url)
        local_filename = f"A1I{month_day_str}0000{month_day_str_f}____0079.RCP91DG1000001"
        t2m_filename = f't2m_{local_filename}.grib'
        
        if os.path.exists(local_filename) or os.path.exists(t2m_filename):
            pass
        else:
            # Make the request to download the file
            response = requests.get(url)
            
            # Check if the request was successful
            if response.status_code == 200:
                # Write the content to the local file
                with open(local_filename, 'wb') as f:
                    f.write(response.content)
                print(f"File downloaded successfully and saved as {local_filename}")
            else:
                print(f"Failed to download file. Status code: {response.status_code}")


        if os.path.exists(t2m_filename):
            print(f"File {t2m_filename} is already there.")
        else:
            data = mv.Fieldset(path=local_filename)
        
            t2m = data.select(shortName='2t')
            
            t2m.write(t2m_filename)
            print(f"Saved {t2m_filename}")
    
            if os.path.exists(local_filename):
              os.remove(local_filename)
            else:
              print("The file does not exist")
            
        fdate = forecast_seq.next(fdate)

8 25
https://xdiss.ecmwf.int/ecpds/home/rcp/20240825/mediumrange/A1I082500000825____0079.RCP91DG1000001
File t2m_A1I082500000825____0079.RCP91DG1000001.grib is already there.
https://xdiss.ecmwf.int/ecpds/home/rcp/20240825/mediumrange/A1I082500000826____0079.RCP91DG1000001
File t2m_A1I082500000826____0079.RCP91DG1000001.grib is already there.
https://xdiss.ecmwf.int/ecpds/home/rcp/20240825/mediumrange/A1I082500000827____0079.RCP91DG1000001
File t2m_A1I082500000827____0079.RCP91DG1000001.grib is already there.
https://xdiss.ecmwf.int/ecpds/home/rcp/20240825/mediumrange/A1I082500000828____0079.RCP91DG1000001
File t2m_A1I082500000828____0079.RCP91DG1000001.grib is already there.
https://xdiss.ecmwf.int/ecpds/home/rcp/20240825/mediumrange/A1I082500000829____0079.RCP91DG1000001
File t2m_A1I082500000829____0079.RCP91DG1000001.grib is already there.
https://xdiss.ecmwf.int/ecpds/home/rcp/20240825/mediumrange/A1I082500000830____0079.RCP91DG1000001
File t2m_A1I082500000830____0079.RCP91DG100000

Each file contains 6 steps for that day: 0, 6, 12 and 18.  
Let's open one file (the last created) to see what we have inside

In [8]:
data = mv.Fieldset(path=t2m_filename)
data.describe()

parameter,typeOfLevel,level,date,time,step,number,paramId,class,stream,type,experimentVersionNumber
2t,surface,0,20240925,0,"144,150,...","0,1,...",167,od,enfh,"cf,pf",0079


We can also list all the fields

In [9]:
data.ls()

,centre,shortName,typeOfLevel,level,dataDate,dataTime,stepRange,dataType,number,gridType
Message,,,,,,,,,,
0,ecmf,2t,surface,0,20110925,0,162,pf,5,regular_ll
1,ecmf,2t,surface,0,20110925,0,162,pf,6,regular_ll
2,ecmf,2t,surface,0,20190925,0,162,pf,3,regular_ll
3,ecmf,2t,surface,0,20110925,0,144,pf,1,regular_ll
4,ecmf,2t,surface,0,20110925,0,162,pf,8,regular_ll
...,...,...,...,...,...,...,...,...,...,...
875,ecmf,2t,surface,0,20230925,0,156,pf,10,regular_ll
876,ecmf,2t,surface,0,20220925,0,156,pf,6,regular_ll
877,ecmf,2t,surface,0,20040925,0,144,pf,2,regular_ll


Next thing we will do is calculate daily mean only over steps.

In [10]:
daily_mean = data.mean(dim=["step"],
    preserve_dims=["shortName", "level", "number", "dataDate"])

In [11]:
daily_mean.ls()

,centre,shortName,typeOfLevel,level,dataDate,dataTime,stepRange,dataType,number,gridType
Message,,,,,,,,,,
0,ecmf,2t,surface,0,20110925,0,162,pf,5,regular_ll
1,ecmf,2t,surface,0,20190925,0,162,pf,5,regular_ll
2,ecmf,2t,surface,0,20120925,0,162,pf,5,regular_ll
3,ecmf,2t,surface,0,20090925,0,162,pf,5,regular_ll
4,ecmf,2t,surface,0,20200925,0,162,pf,5,regular_ll
5,ecmf,2t,surface,0,20100925,0,162,pf,5,regular_ll
6,ecmf,2t,surface,0,20150925,0,162,pf,5,regular_ll
7,ecmf,2t,surface,0,20170925,0,162,pf,5,regular_ll
8,ecmf,2t,surface,0,20140925,0,162,pf,5,regular_ll


In [12]:
percentiles100 = list(range(101))
percentiles = [1, 10, 50, 90, 99]
pc = mv.percentile(data=daily_mean, percentiles=percentiles)
pc.ls()

,centre,shortName,typeOfLevel,level,dataDate,dataTime,stepRange,dataType,number,gridType
Message,,,,,,,,,,
0,ecmf,2t,surface,0,20110925,0,162,pf,5,regular_ll
1,ecmf,2t,surface,0,20190925,0,162,pf,5,regular_ll
2,ecmf,2t,surface,0,20120925,0,162,pf,5,regular_ll
3,ecmf,2t,surface,0,20090925,0,162,pf,5,regular_ll
4,ecmf,2t,surface,0,20200925,0,162,pf,5,regular_ll


In [13]:
data_area = [50,10,40,30]
margins = [2, -2, -2, 2]
view_area = [a + b for a, b in zip(data_area, margins)]

In [14]:
coastlines = mv.mcoast(map_coastline_land_shade=True,
                       map_coastline_land_shade_colour="RGB(0.85,0.85,0.85)",
                       map_coastline_sea_shade=True,
                       map_coastline_sea_shade_colour="RGB(0.95,0.95,0.95)",)
view = mv.geoview(map_area_definition="corners", area=view_area, coastlines=coastlines)
cont_auto = mv.mcont(legend=True,     
                     contour_automatic_setting = "style_name",
                     contour_style_name        = "sh_mc_t_fM80t60",
                     grib_scaling_of_derived_fields=True)

In [15]:
mv.plot(view, pc, cont_auto)

Image(value=b'', layout="Layout(visibility='hidden')")

Label(value='Generating plots....')

Now finally we can do it for the whole dataset of 7 days forecast.

In [16]:
clim_dates = sequence.bracket(clim_date,4,strict=False)
for date in clim_dates:
    print(date)

2024-08-25
2024-08-29
2024-09-01
2024-09-05
2024-09-09
2024-09-13
2024-09-17
2024-09-21
2024-09-25
